In [ ]:
import json
import base64
from requests_toolbelt.multipart import decoder
import re
from base64 import b64encode as enc64
from base64 import b64decode as dec64
import PIL
from PIL import Image
import io
import boto3

# Парсер для form-data
def get_field_name(content_disposition):
    parts = content_disposition.split(';')
    for part in parts:
        part_stripped = part.strip()
        search_result = re.search("^name=\"(.*)\"$", part_stripped)
        
        if search_result:
            return search_result.group(1)

# Функция, которая декодирует байты
def encode_file(file):
    return base64.b64encode(file)

def handler(event, context):
    # aws_access_key_id == "m5zYttENO7OmoLKbTFJy"
    # aws_secret_access_key == "ReN4DyzoVOUzI4OU6UGEQiKPBEisZhbn3cz997JH" 
    
    body = event['body']
    content_type = event['headers']['Content-Type']
    decoded_body = base64.b64decode(body)
        
                                
    for part in decoder.MultipartDecoder(decoded_body, content_type).parts: 
        content_disposition = part.headers[b'Content-Disposition'].decode('utf-8');
        field_name = get_field_name(content_disposition)
        file = part.content
         
    image = encode_file(file)

    session = boto3.session.Session()
    s3 = session.client(
            service_name='s3',
            endpoint_url='https://storage.yandexcloud.net',
            aws_access_key_id = "KLoEu90zbSgVz4KzQCjB",
            aws_secret_access_key = "fQs4-sMM211NqPMn3uSXoHnz3E38JeCoPaEy5D_P"

    )       
    s3.put_object(Bucket='image-handler-bucket', Key='sample2', Body='TEST', StorageClass='STANDARD')
    # some_binary_data = part.content
    some_binary_data = image
    s3 = boto3.resource('s3')
    object = s3.Object('image-handler-bucket', 'sample2.jpeg')
    object.put(Body = some_binary_data)

    # достаем файл из бакета
    get_object_response = s3.get_object(Bucket='image-handler-bucket',Key='sample2')#в скобках имя бакета и имя файла, в бакете
    print( get_object_response['Body'].read())
    
    return {
        'statusCode': 200,
        'headers': {
        'content-type': 'image/png'
        },
        'body': image.decode('utf-8'),
        'isBase64Encoded': True
        }

In [ ]:
Credentials:
aws_access_key_id = KLoEu90zbSgVz4KzQCjB
aws_secret_access_key = fQs4-sMM211NqPMn3uSXoHnz3E38JeCoPaEy5D_P